In [1]:
import os
import shutil
import sys
sys.path.append('../datapipeline')
sys.path.append('../imgclsmob/tensorflow2')
sys.path.append('../akhelpers')


import autokeras as ak
from Common_AK import CustomGlobalPool
from Resnet_AK import CustomResnetBlock

from DatagenClassification import TFClassificationDataGenerator
import random
random.seed(47)
import albumentations as A
import numpy as np
import tensorflow as tf
from kerastuner.engine.hyperparameters import HyperParameters

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
IMAGE_SIZE = (None, None)
NUM_CHANNELS = (3,)
NUM_CLASSES = 3
MAX_NETWORK_SEARCH_TRIALS = 500
BATCH_SIZE = 1
VAL_DS_SIZE = 2058
TRAIN_DS_SIZE = 4464
EPOCHS = 10
MODEL_DIR = 'auto_model'
PROJECT_NAME = 'resnet_ak'
OVERITE_PROJECT = False

train_files = ['../../../data/coco_person_classification_tiny/train_class-00000-of-00005', 
               '../../../data/coco_person_classification_tiny/train_class-00001-of-00005', 
               '../../../data/coco_person_classification_tiny/train_class-00002-of-00005', 
               '../../../data/coco_person_classification_tiny/train_class-00003-of-00005', 
               '../../../data/coco_person_classification_tiny/train_class-00004-of-00005']
val_files = ['../../../data/coco_person_classification_tiny/val_class-00000-of-00005', 
             '../../../data/coco_person_classification_tiny/val_class-00001-of-00005', 
             '../../../data/coco_person_classification_tiny/val_class-00002-of-00005', 
             '../../../data/coco_person_classification_tiny/val_class-00003-of-00005', 
             '../../../data/coco_person_classification_tiny/val_class-00004-of-00005']
has_bg = True

In [3]:
# restric the search space
hp = HyperParameters()
hp.Choice(
            "optimizer",
            ["adam", "sgd"],
            default="adam",
        )
# hp.Fixed(
#             "optimizer",
#             "sgd"
#         )

'adam'

### Define Model

In [4]:
input_node = ak.Input()
output_node = ak.ImageAugmentation(
    translation_factor=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    rotation_factor=1.0,
    zoom_factor=0.2,
    contrast_factor=0.2
)(input_node)
output_node = ak.Normalization()(output_node)
output_node = CustomResnetBlock(in_size=IMAGE_SIZE)(output_node)
output_node = CustomGlobalPool()(output_node)
output_node = ak.ClassificationHead(num_classes=NUM_CLASSES, multi_label=True)(output_node)
auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    overwrite=OVERITE_PROJECT,
    directory=MODEL_DIR,
    max_trials=MAX_NETWORK_SEARCH_TRIALS,
    hyperparameters=hp,
    project_name=PROJECT_NAME)

### Define Image Preprocessing 

In [5]:
train_transform = A.Compose([
        A.Rotate(limit=180),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
#         A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=0, p=0.2),
#         A.OneOf([
#             A.OpticalDistortion(p=0.3),
#             A.GridDistortion(p=.1),
#             A.IAAPiecewiseAffine(p=0.3),
#         ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ])

val_transform = A.Compose([
        A.Rotate(limit=180),
        A.Flip(),
#         A.ToGray(0.2),
    ])

### Dataset define and process

In [6]:
df_gen = TFClassificationDataGenerator(train_files=train_files,
                                       val_files=val_files,
                                       num_classes=NUM_CLASSES,
                                       train_size=TRAIN_DS_SIZE,
                                       val_size=VAL_DS_SIZE,
                                       train_aug=None,
                                       val_aug=None,
                                       repeatds=False
                                      )

train_gen = df_gen.train_generator(TRAIN_DS_SIZE)
val_gen = df_gen.validation_generator(VAL_DS_SIZE)

In [7]:
# train_dataset_mem = []
# for x, y in train_gen:
# #             x, y = next(iter(ds))
#     x1 = tf.squeeze(x)
#     y1 = tf.squeeze(y)
#     train_dataset_mem.append((x1, y1))
    
val_dataset_mem = []
for x, y in val_gen:
#             x, y = next(iter(ds))
    x1 = tf.squeeze(x)
    y1 = tf.squeeze(y)
    val_dataset_mem.append((x1, y1))

In [8]:
def get_data_generator(ds, n_batch):
    """Get a generator returning n_batches random data of batch_size with n_features."""

    def data_generator():
        for x, y in ds:
#             x, y = next(iter(ds))
#             x1 = tf.squeeze(x)
#             y1 = tf.squeeze(y)
            yield x, y

    return data_generator

# train_dataset = tf.data.Dataset.from_generator(
#     get_data_generator(train_dataset_mem,TRAIN_DS_SIZE),
#     output_types=(tf.float32, tf.uint8),
#     output_shapes=(IMAGE_SIZE + NUM_CHANNELS, (NUM_CLASSES,)),
# ).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_generator(
    get_data_generator(val_dataset_mem, VAL_DS_SIZE),
    output_types=(tf.float32, tf.uint8),
    output_shapes=(IMAGE_SIZE + NUM_CHANNELS, (NUM_CLASSES,)),
).batch(BATCH_SIZE)

### Train Model

In [9]:
class ClearTrialCheckpoints(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        dir_to_look = os.path.join(MODEL_DIR, PROJECT_NAME)
        dir_ls = [os.path.join(dir_to_look, d) for d in os.listdir(dir_to_look) if os.path.isdir(os.path.join(dir_to_look, d)) and 'trial' in d]
        for d in dir_ls:
            dir_of_concern = os.path.join(d, 'checkpoints')
            if os.path.isdir(dir_of_concern):
                print(dir_of_concern)
                shutil.rmtree(dir_of_concern)


In [10]:
callbacks = [EarlyStopping(patience=3),
            ReduceLROnPlateau(patience=1),
             ClearTrialCheckpoints(),
            ]

In [ ]:
# auto_model.fit(train_dataset, validation_data=val_dataset, batch_size=None, epochs=EPOCHS, callbacks=callbacks)
auto_model.fit(val_dataset, batch_size=None, epochs=EPOCHS, callbacks=callbacks)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
optimizer         |adam              |?                 
custom_resnet_b...|False             |?                 
custom_resnet_b...|0                 |?                 
custom_resnet_b...|1.4               |?                 
custom_global_p...|1                 |?                 
classification_...|0                 |?                 
learning_rate     |0.001             |?                 

Epoch 1/10
      8/Unknown - 13s 2s/step - loss: 0.7858 - accuracy: 0.5000

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-2066427a1172>", line 2, in <module>
    auto_model.fit(val_dataset, batch_size=None, epochs=EPOCHS, callbacks=callbacks)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/auto_model.py", line 275, in fit
    **kwargs
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/engine/tuner.py", line 173, in search
    super().search(epochs=epochs, callbacks=new_callbacks, **fit_kwargs)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/kerastuner/engine/base_tuner.py", line 131, in search
    self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/sahil/.local/share/vir

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-2066427a1172>", line 2, in <module>
    auto_model.fit(val_dataset, batch_size=None, epochs=EPOCHS, callbacks=callbacks)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/auto_model.py", line 275, in fit
    **kwargs
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/engine/tuner.py", line 173, in search
    super().search(epochs=epochs, callbacks=new_callbacks, **fit_kwargs)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/kerastuner/engine/base_tuner.py", line 131, in search
    self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/sahil/.local/share/vir

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-2066427a1172>", line 2, in <module>
    auto_model.fit(val_dataset, batch_size=None, epochs=EPOCHS, callbacks=callbacks)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/auto_model.py", line 275, in fit
    **kwargs
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/autokeras/engine/tuner.py", line 173, in search
    super().search(epochs=epochs, callbacks=new_callbacks, **fit_kwargs)
  File "/home/sahil/.local/share/virtualenvs/network_search-ZkVDDU0n/lib/python3.7/site-packages/kerastuner/engine/base_tuner.py", line 131, in search
    self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/sahil/.local/share/vir

### Show Model 

In [ ]:
model = auto_model.export_model()
model.summary()

In [ ]:
auto_model.tuner.results_summary()

In [ ]:
dict(auto_model.tuner.get_best_hyperparameters(num_trials=1)[0])